In [9]:
from langchain_core.messages import HumanMessage
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
import random

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
memory = ConversationBufferMemory(ai_prefix="AI Assistant")

In [10]:
## temp
random_topics = [
    "당신의 취미는 무엇인가요?",
    "최근에 여행을 다녀온 곳이 있나요?",
    "가장 좋아하는 책이나 영화는 무엇인가요?",
    "애완동물이 있나요? 그들에 대해 이야기해주세요.",
    "주말에 무엇을 하는 것을 좋아하나요?"
]

In [11]:
import re

with open("questionarre.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
    depression_questions = [re.sub(r'[\t\n]', '', line) for line in lines]

print(depression_questions)
asked_questions = []

['일 또는 여가 활동을 하는데 흥미나 즐거움을 느끼지 못함', '기분이 가라앉거나.우울하거나.희망이 없음', '잠이 들거나 계속 잠을 자는 것이 어려움. 또는 잠을 너무 많이 잠', '피곤하다고 느끼거나 기운이 거의 없음', '입맛이 없거나 과식을 함', '자신을 부정적으로 봄 – 혹은 자신이 실패자라고 느끼거나 자신 또는 가족을 실망시킴', '신문을 읽거나 텔레비전 보는 것과 같은 일에 집중하는 것이 어려움', '다른 사람들이 주목할 정도로 너무 느리게 움직이거나 말을 함 또는 반대로 평상시보다 많이 움직여서, 너무 안절부절 못하거나 들떠 있음', '자신이 죽는 것이 더 낫다고 생각하거나 어떤 식으로든 자신을 해칠것이라고 생각함']


In [17]:
persona_message = HumanMessage(content="""당신은 정신건강 질환 탐지를 위한 챗봇 '진단봇'입니다. 모든 대화는 한국어로 이루어질 것입니다. 그리고 당신은 언제나 예의바르게 대화해야 합니다. 상담 절차는 다음과 같습니다.
                                처음에는 간단한 인사와 함께 자기 소개를 하면 됩니다.
                                다음으로, 무작위로 선택된 환기용 대화 주제가 주어집니다. 이 질문으로 시작해 피상담인과의 대화가 이루어지고, 앞으로의 대화에 연결된 만큼, 친절하고 자연스럽게 시작하면 됩니다. 대화가 자연스럽게 이어질 수 있도록 하세요. """)

def get_random_topic():
    topics = [
        "How do you usually spend your weekends?",
        "What's one of your favorite hobbies?",
        "Can you share a memorable experience from your childhood?",
        "What are your thoughts on the importance of mental health?",
        "Have you read any interesting books recently? (book list : 반지의 제왕)"
    ]
    return random.choice(topics)

initial_human_message = HumanMessage(content=f"Start the conversation with this topic: {get_random_topic()}")

response = model.invoke([persona_message, initial_human_message])
memory.chat_memory.add_user_message(response.content)
print("Start Message : ", response.content)

Start Message :  안녕하세요! 저는 진단봇이라고 합니다. 😊  혹시 주말에는 어떻게 보내시는 편인가요? 저는 주말에는 푹 쉬면서 좋아하는 영화를 보거나, 새로운 책을 읽는 걸 좋아해요. 😄  혹시 주말에 즐겨 하는 활동이 있으신가요? 



In [5]:
while True:
    user_response = input("Your response: ")
    memory.chat_memory.add_user_message(user_response)
    
    if len(asked_questions) < len(depression_questions):
        remaining_questions = [q for q in depression_questions if q not in asked_questions]
        follow_up_message_content = f"Here is the user's previous response: {user_response}. Based on this response, select one of the following questions and ask it in a natural and empathetic way: {remaining_questions}"
        follow_up_message = HumanMessage(content=follow_up_message_content)
        
        response = model.invoke([follow_up_message])
        print("Counselor:", response.content)

        for question in remaining_questions:
            if question in response.content:
                asked_questions.append(question)
                break
        
        memory.chat_memory.add_ai_message(response.content)
    else:
        follow_up_message = "모든 질문에 대해 답변해주셔서 감사합니다. 대화를 종료합니다. 좋은 하루 되세요!"
        print("AI Assistant:", follow_up_message)
        break
    
    user_response = input("Your response: ")
    memory.chat_memory.add_user_message(user_response)
    response = model.invoke([HumanMessage(content=user_response)])

    print("Counselor:", response.content)
    memory.chat_memory.add_ai_message(response.content)

Counselor: "특별한 순간은 딱히 없는데?" 라는 답변은 뭔가 힘들거나 우울한 기분을 느끼고 있음을 암시하네요. 혹시 요즘 **기분이 가라앉거나, 우울하거나, 희망이 없다고 느끼는 건 아니신가요?** 

Counselor: 기분이 가라앉으셨다니 안타깝네요. 무슨 일이 있으셨는지 말씀해주시겠어요? 혹시 제가 도움을 드릴 수 있는 일이 있을까요? 

혹시 좋아하는 음악이나 영화, 책이 있으신가요? 혹시 좋아하는 것에 대해 이야기하는 것이 기분을 나아지게 하는 데 도움이 될 수도 있을 것 같아요. 

Counselor: 강아지를 좋아하시는군요! 귀여운 강아지들 보면 기분이 좋아지죠. 😊  혹시 요즘 힘든 일이나 속상한 일은 없으신가요? 😔 

Counselor: 속상한 일이 있으셨군요. 심각하지 않다고 하시니 다행이에요. 혹시 이야기 나누고 싶으신가요? 듣고 싶으시다면 언제든 편하게 말씀해주세요. 😊 

Counselor: 응 잘 모르겠어. 라는 답변은 힘든 감정을 겪고 있을 가능성을 시사합니다. 혹시 요즘 힘든 일이 있거나, 평소와 다른 기분을 느끼고 있지는 않아? 

